# 株価を分析する

In [ ]:
# コード/ライブラリをインポート

import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas_datareader
import sklearn
import sklearn.linear_model
import sklearn.model_selection
import yfinance

yfinance.pdr_override()

## データウェアハウス

In [ ]:
# 13-6-1/株価のデータを取得
df_aapl = pandas_datareader.data.get_data_yahoo('AAPL', '2020-01-01')
df_meta = pandas_datareader.data.get_data_yahoo('META', '2020-01-01')
df_gold = pandas_datareader.data.get_data_yahoo('GLD', '2020-01-01')
df_aapl.tail(3)

## データに対して統計学的な処理をする

In [ ]:
# 13-6-2/株価と単純移動平均をグラフ化
df_aapl['SMA'] = df_aapl['Close'].rolling(window=14).mean()
df_aapl['Close'].plot(figsize=(15,6), color='red')
df_aapl['SMA'].plot(figsize=(15,6), color='green')
plt.show()

## データマイニング

In [ ]:
# 13-6-3/Apple社とMeta社の株価を比較
df_aapl['Close'].plot(figsize=(15,6), color='red')
df_meta['Close'].plot(figsize=(15,6), color='blue')
plt.show()

In [ ]:
# 13-6-4/株価の変化率を求める
df_aapl['changing'] = (((df_aapl['Close'] - df_aapl['Open'])) / (df_aapl['Open']) * 100)
df_meta['changing'] = (((df_meta['Close'] - df_meta['Open'])) / (df_meta['Open']) * 100)
df_gold['changing'] = (((df_gold['Close'] - df_gold['Open'])) / (df_gold['Open']) * 100)
df_aapl.tail(2).round(2)

In [ ]:
# 13-6-5/３つの株価の変化率をグラフにする
df_aapl['changing'].tail(100).plot(grid=True, figsize=(15,6), color='red')
df_meta['changing'].tail(100).plot(grid=True, figsize=(15,6), color='blue')
df_gold['changing'].tail(100).plot(grid=True, figsize=(15,6), color='orange')
plt.show()

# 株価を予測する

## 機械学習のためのデータを用意する

In [ ]:
# 13-6-6/終値を格納する
df_aapl['label'] = df_aapl['Close'].shift(-30)
df_aapl.tail(35)

## 線形回帰で株価を予測する

In [ ]:
# 13-6-7/学習させるデータをXに格納
X = np.array(df_aapl.drop(['label', 'SMA'], axis='columns'))
X = sklearn.preprocessing.scale(X)
X

In [ ]:
# 13-6-8/正解のデータをyに格納
y = np.array(df_aapl['label'])
y

In [ ]:
# コード/訓練用データとテスト用データに分割
y = y[:-30]
X = X[:-30]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

# 13-6-9/機械学習オブジェクトを作成して学習させる
lr = sklearn.linear_model.LinearRegression()
lr.fit(X_train, y_train)
accuracy = lr.score(X_test, y_test)
accuracy

In [ ]:
# 13-6-10/過去30日間から未来の30日間を予測
predicted_data = lr.predict(X[-30:])
predicted_data

In [ ]:
# 13-6-11/未来の株価の予測をグラフにする
df_aapl['Predict'] = np.nan

last_date = df_aapl.iloc[-1].name
one_day = 86400
next_day = last_date.timestamp() + one_day

for data in predicted_data:
    next_date = datetime.datetime.fromtimestamp(next_day)
    df_aapl.loc[next_date] = np.append([np.nan] * (len(df_aapl.columns)-1), data)
    next_day += one_day

df_aapl['Close'].plot(figsize=(15,6), color='green')
df_aapl['Predict'].plot(figsize=(15,6), color='orange')
plt.show()

## Point/深層学習、人工知能

In [ ]:
# 13-6-12/株を買うべきか売るべきか判断する
stock_after_a_month = df_aapl['Predict'][-1]
stock_today = df_aapl['Close'][-31]
if stock_after_a_month > stock_today:
    print('Buy Now.')
else:
    print('Sell Now.')